In [2]:
import lava.lib.dl.slayer as slayer
import matplotlib.pyplot as plt
import numpy as np
import nir
# import norse
import tonic
import os
import torch
import tqdm

import sys
sys.path.append("/Users/steve/Code/NIR/paper")

from nir_to_lava import ImportConfig, LavaLibrary, import_from_nir

In [2]:
graph = nir.read('cnn_sinabs.nir')
import_config = ImportConfig(library_preference=LavaLibrary.LavaDl)
net = import_from_nir(graph, import_config)

In [3]:
bs = 128
collate = tonic.collation.PadTensors(batch_first=False)
to_frame = tonic.transforms.ToFrame(sensor_size=tonic.datasets.NMNIST.sensor_size, time_window=1e3)
test_ds = tonic.datasets.NMNIST("./nmnist", transform=to_frame, train=False)
test_dl = torch.utils.data.DataLoader(test_ds, shuffle=True, batch_size=bs, collate_fn=collate)

In [6]:
accuracies = []
pbar = tqdm.tqdm(total=len(test_dl), desc="Processing")

for idx, (x, y) in enumerate(test_dl):
    x = torch.moveaxis(x, 0, -1)
    out = net(x)
    pred = out.mean(axis=-1).argmax(axis=1)
    accuracy = (pred == y).sum() / x.shape[0]
    accuracies.append(accuracy)
    pbar.set_postfix(accuracy="{:.2f}%".format(sum(accuracies)/len(accuracies)*100))
    pbar.update(1)
pbar.close()
accuracies = np.array(accuracies)
print(f'accuracy: {accuracies.mean():.2%} +/- {accuracies.std():.2%}')

Processing: 100%|██████████| 79/79 [17:19<00:00, 13.16s/it, accuracy=98.08%]

accuracy: 98.08% +/- 1.20%


In [ ]:
# compare activations and accuracy with stored ones
if os.path.exists("lava_accuracy.npy"):
  stored_acc = np.load("lava_accuracy.npy")
  assert np.allclose(stored_acc, accuracies.mean(), atol=1e-3)

if os.path.exists("lava_activity.npy"):
  stored_activations = np.load("lava_activity.npy")
  numbers = np.load("cnn_numbers.npy")
  with torch.no_grad():
    y = net.blocks[0](torch.moveaxis(torch.tensor(numbers), 0, -1).float())
    out = net.blocks[1](y).moveaxis(-1, 0)
    out = out.numpy()
    assert np.allclose(stored_activations, out, atol=1e-3)

## Write results to disk

In [5]:
np.save("lava_accuracy.npy", accuracies.mean())

In [123]:
numbers = np.load("cnn_numbers.npy")
with torch.no_grad():
  y = net.blocks[0](torch.moveaxis(torch.tensor(numbers), 0, -1).float())
  out = net.blocks[1](y).moveaxis(-1, 0)
  np.save("Lava_activity.npy", out.numpy())
out.shape

torch.Size([300, 10, 16, 16, 16])